# Хеширование признаков

Рассмотрим прием с хешированием и изучим инструменты, которые успешно его применяют.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("airbnb-100k.tsv", delimiter='\t')

In [3]:
df.head()

,Price,Description
0,50.0,"Hi everyone, Cosy bedroom in a modern apartmen..."
1,125.0,"Very comfortable and calm apartment, 2 rooms i..."
2,59.0,"At a few minutes by walking from République, O..."
3,50.0,Come stay here in my little nest. It's a very ...
4,48.0,"Studio de 25 m2, idéalement situé au centre de..."


Задачу будем решать такую же - предсказываем цену апартаментов по текстовому описанию. 

### Vowpal Wabbit

Vowpal Wabbit - продвинутый инструмент, оптимизированный для обучения линейных моделей на больших объемах данных. Он активно использует хеширование признаков для своей работы. 

Это инструмент командной строки, который доступен в виде команды `vw`.

Полную информацию можно найти на <a href="https://vowpalwabbit.org/">официальном сайте</a>

In [4]:
! vw --help | head

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
driver:
  --onethread           Disable parse thread
VW options:
  --ring_size arg (=256, ) size of example ring
  --strict_parse           throw on malformed examples
Update options:
  -l [ --learning_rate ] arg Set learning rate
  --power_t arg              t power value
  --decay_learning_rate arg  Set Decay factor for learning_rate between passes
  --initial_t arg            initial t value


### Формат данных

VW для обучения требует свой особенный формат хранения данных. Он выглядит следующим образом: 

```
[Label] [Importance] [Tag]|Namespace Features |Namespace Features ... |Namespace Features
```

* Label - значение целевой переменной. Если не указывать, объект не будет использоваться в обучении
* Importance - вес объекта. Если не указывать, равен 1
* Tag - пометка объекта. Никак не влияет на процесс обучения, но добавляет семантики и "читаемости" данных для человека
* Namespace - название области признаков. Используется, чтобы разные по сути признаки с одинаковым названием не пересекались
* Features - признаки. Это или пара <название признака>:<значение признака> или просто <название признака>. В последнем случае будет считаться, что значение равно 1

Например, попробуем закодировать несколько элементов из Iris в этом формате

| Длина чашелистика | Ширина чашелистика | Длина лепестка | Ширина лепестка |  Вид ириса |
|:-----------------:|:------------------:|:--------------:|:---------------:|:----------:|
| 5.1               | 3.5                | 1.4            | 0.2             | setosa     |
| 4.9               | 3.0                | 1.4            | 0.2             | setosa     |
| 7.0               | 3.2                | 4.7            | 1.4             | versicolor |

Будем считать setosa классом 1, а versicolor классом -1.

In [5]:
%%writefile iris_1.example.vw
1 | sepal_length:5.1 sepal_width:3.5 petal_length:1.4 petal_width:0.2
1 | sepal_length:4.9 sepal_width:3.0 petal_length:1.4 petal_width:0.2
-1 | sepal_length:7.0 sepal_width:3.2 petal_length:4.7 petal_width:1.4

Writing iris_1.example.vw


В данном наборе мы дополнительно можем сгруппировать признаки - признаки для чашелистика и признаки для лепестка. Это позволит логически их разделить, сэкономит место, а также позволит в дальнейшем оперировать этими группами.

Запишем точно эти же данные, но с использованием именованных секций.

In [6]:
%%writefile iris_2.example.vw
1 |sepal length:5.1 width:3.5 |petal length:1.4 width:0.2
1 |sepal length:4.9 width:3.0 |petal length:1.4 width:0.2
-1 |sepal length:7.0 width:3.2 |petal length:4.7 width:1.4

Writing iris_2.example.vw


Подобный формат записи данных позволяет очень удобно записывать категориальные признаки в формате One-Hot Encoding. При таком кодировании мы бы для признака, соответствующего конкретной категории, выставили бы значение 1, а всем остальным - 0.

Например для признака с полом человека превратится в два признака

| gender_man | gender_woman |
|:----------:|:------------:|
| 1          | 0            |

Если же в VW не указывать явно значение признака, то оно будет в автоматическом режиме выставлено равным 1, чего мы и ожидаем при One-Hot кодировании.

Попробуем для примера записать вот такие данные в этом формате

| Класс |  Цвет фона | Включена ли темная тема | Размер шрифта | Межстрочный интервал |
|:-----:|:----------:|:-----------------------:|:-------------:|:--------------------:|
| 1     | White      | да                      | 12            | 1.5                  |
| -1    | Black      | нет                     | 14            | 1.5                  |
| 1     | White, Red | нет                     | 12            | 2                    |
| -1    | Black, Red | да                      | 15            | 1.5                  |

In [7]:
%%writefile design.example.vw
1 |color white |theme dark |font size:12.0 interval:1.5
-1 |color black |theme normal |font size:14.0 interval:1.5
1 |color white red |theme normal |font size: 12.0 interval:2.0
-1 |color black red |theme dark |font size: 15.0 interval:1.5

Writing design.example.vw


Можно заметить, что такой подход к кодированию также позволяет очень просто кодировать текстовые признаки в формате bag-of-words. При использовании bag-of-words мы для каждого слова добавляет отдельный бинарный признак, который равен 1, если слово есть в тексте.

Таким образом, чтобы закодировать в формате vw какой-то текст через bag-of-words достаточно... Просто написать этот текст! Единственное, что необходимо сделать дополнительно - это очистить сам текст от лишних символов, таких как знаки препинания, удаления, кавычки и так далее. Очищенный текст - это и есть кодирование признаков в vw.

Попробуем, например, закодировать вот такой набор данных.

| Оценка |                             Заголовок                              |                                                                      Комментарий                                                                     |
|:------:|:------------------------------------------------------------------:|:----------------------------------------------------------------------------------------------------------------------------------------------------:|
| 2      | To lazy to watch film                                              | My family and I normally do not watch local movies for the simple reason that they are poorly made                                                   |
| 5      | Well-directed and fairly notorious piece of Italian nunsploitation | This file is a genuinely moving and intelligent movie with plenty of nudity and gore.You can't go wrong with it.5 out of 5.                         |
| 3      | this was at one time the worst movie I had ever seen               | But since than time, I have seen many more movies that are worse (how is it possible??) Therefore, to be fair, I had to give this movie a 3 out of 5 |

In [8]:
%%writefile reviews.vw
2.0 |title to lazy to watch film |review my family and i normally do not watch local movies for the simple reason that they are poorly made
5.0 |title well directed and fairly notorious piece of italian nunsploitation |review this file is a genuinely moving and intelligent movie with plenty of nudity and gore you can t go wrong with it 8 out of 10 
3.0 |title this was at one time the worst movie i had ever seen |review but since than time  i have seen many more movies that are worse how is it possible  therefore to be fair  i had to give this movie a 3 out of 5 

Writing reviews.vw


In [9]:
! cat reviews.vw

2.0 |title to lazy to watch film |review my family and i normally do not watch local movies for the simple reason that they are poorly made
5.0 |title well directed and fairly notorious piece of italian nunsploitation |review this file is a genuinely moving and intelligent movie with plenty of nudity and gore you can t go wrong with it 8 out of 10 
3.0 |title this was at one time the worst movie i had ever seen |review but since than time  i have seen many more movies that are worse how is it possible  therefore to be fair  i had to give this movie a 3 out of 5 


В этом примере именованные секции играют важную роль, так как одно и тоже слово может оказаться и в заголовке и в тексте комментария. Без секций мы бы считали это одним признаком. Секции позволяют нам разделять слова, которые использовались в заголовке от слов, которые использовались в описании.

Итак, мы научились формировать данные для VW, самое время запустить его. Попробуем использовать наши примеры.

In [10]:
! vw reviews.vw --final_regressor reviews_result.vw.bin

final_regressor = reviews_result.vw.bin
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = reviews.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
4.000000 4.000000            1            1.0   2.0000   0.0000       25
13.653724 23.307447            2            2.0   5.0000   0.1722       36

finished run
number of examples = 3
weighted example sum = 3.000000
weighted label sum = 10.000000
average loss = 10.574909
best constant = 3.333333
total feature number = 106


На экране можно видеть отчет обучения, а в файл `reviews_result.vw.bin` записались веса модели. По-умолчанию используется линейная регрессия с MSE.

Для того, чтобы получить предсказания нет отдельной команды. Это связано с особым форматом работы инструмента. Можно считать, что vw всегда запускается в режиме обучения, однако есть несколько флагов, которые умеют контролировать этот процесс.

Схематично это можно изобразить на картинке

<img src="img/vw.png">

Таким образом при каждом запуске VW получает входные данные, с которыми он сейчас будет работать. Далее он начинает обучение на этих данных. Если указан `--initial_regressor` то он возьмет указанные веса в качестве начальных - то есть попытается "дообучить" модель, которую мы указали. Параллельно с этим для каждого пройденного элемента он производит предсказание. Если указать параметр `--predictions`, то эти данные он отдельно запишет в файл. По окончанию обучения новые полученные веса он попробует сохранить в файл, указанный через `--final_regressor`. И также есть очень важный флаг `--testonly`, говорящий о том, что обновлять веса (то есть обучаться) в процессе не требуется.

Подобная схема позволяет более гибко задавать режимы работы. Вот возможные сценарии работы, которые можно выставить этими ключами.

* `--final_regressor` - просто обучаем модель на данных
* `--testonly`, `--initial_regressor`, `--predictions` - просто делаем предсказания модели на данных
* `--final_regressor`, `--initial_regressor` - дообучаем модель на новых данных

И так далее. Попробуем сделать предсказание на нашем же примере.

In [11]:
! vw --testonly --initial_regressor reviews_result.vw.bin --predictions reviews_prediction.txt reviews.vw

only testing
predictions = reviews_prediction.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = reviews.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.042279 0.042279            1            1.0   2.0000   1.7944       25
2.310258 4.578236            2            2.0   5.0000   2.8603       36

finished run
number of examples = 3
weighted example sum = 3.000000
weighted label sum = 10.000000
average loss = 1.583771
best constant = 3.333333
total feature number = 106


In [12]:
! cat reviews_prediction.txt

1.794381
2.860319
2.638340


Мы научились базовым приемам для работы с VW. Попробуем теперь обучить модель для решения оригинальной задачи. 

Для начала необходимо привести данные в нужный формат.

In [13]:
import re

def convert_to_vw(raw_text, target):
    word_pattern = re.compile(r"[a-zA-Z0-9_]+")
    words = []
    for match in re.finditer(word_pattern, raw_text.lower()):
        words.append(match.group(0))
    
    if not words: 
        return None
    return "{} |d {}".format(float(target), " ".join(words))
    

In [14]:
example = convert_to_vw(df['Description'][0], df['Price'][0])
print(example)

50.0 |d hi everyone cosy bedroom in a modern apartment located in a central area paris 11th the apartment it s a 2 bedrooms one is mine apartment of 47m2 509 sq ft fully renovated warm atmosphere a living room with a equiped kitchen wifi i provide towels and sheets central area cosmopolite non touristic very close to the marais bastille and republic transports 2 metro 3 walk saint ambroise line 9 or richard lenoir line 5 2 city bike station best j


In [15]:
df.dropna(subset=['Price', "Description"], inplace=True)
Y = df['Price']
X = df['Description']

In [16]:
def write_vw(X_data, Y_data, filename):
    with open(filename, "w") as f:
        for x, y in zip(X_data, Y_data):
            vw_object = convert_to_vw(x, y)
            if not vw_object:
                continue
            f.write(vw_object + '\n')

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=100)

In [19]:
write_vw(X_train, y_train, "airbnb-train.vw")
write_vw(X_test, y_test, "airbnb-test.vw")

In [20]:
! head -n 1 airbnb-train.vw

497.0 |d bright cozy and simple 2 bedroom apartment with balcony furnished rental on amager in quiet and green surroundings close to the dr city concerthall and university amager f lled park and with several bathing options both at island s brygge and amager beach park 10 15min walking distance from amagerbro and dr city metro st which has good transport options the apartment is 47kvm with furnished living room bedroom kitchen bathroom and toilet wifi stereo tube amplifier with vinyl collection and a b o television tube stove fridge and basic kitchenware half of the living room has been designed for use as a studio workshop drawing painting and there is a little storage in the bedroom the apartment is simple with good details quiet neighbors and no noise from cars suitable for a person or couple the apartment is located at amager close to restaden and islands brygge the new dr building denmark s radio and television station with its large concert hall is 7 mi


In [21]:
! head -n 1 airbnb-test.vw

67.0 |d this holiday apartment in malaga with wifi is ideal for those who want to live in a quiet area close to the beach and malaga city center the accommodation offers a modern and stylish living room with dining table sofa armchair and a flat screen tv this holiday apartment in malaga with wifi is ideal for those who want to live in a quiet area close to the beach and malaga city center the accommodation offers a modern and stylish living room with dining table sofa armchair and a flat screen tv there is access to the private terrace a large fully equipped kitchen a bathroom with bath one bedroom with fitted wardrobes and double or single and another bedroom with fitted wardrobes double bed and en suite bathroom with shower we offer parking in the same building for 15 per day you can use the communal swimming pool from may to october optional services not included in the price garage price 15 per day minimum 60 cot crib price 5 per day minimum 20


In [22]:
! cat airbnb-train.vw | wc -l

65969


In [23]:
! cat airbnb-test.vw | wc -l

32482


Обучаем модель

In [24]:
%%time

! vw --final_regressor airbnb-lin-model-1.vw.bin airbnb-train.vw

final_regressor = airbnb-lin-model-1.vw.bin
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-train.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
247009.000000 247009.000000            1            1.0 497.0000   0.0000      167
124369.634766 1730.269531            2            2.0  42.0000   0.4035      185
73513.864502 22658.094238            4            4.0 198.0000   0.2531        9
67610.261047 61706.657593            8            8.0  49.0000   3.8943       47
48401.856529 29193.452011           16           16.0  45.0000  13.4168      152
33535.183784 18668.511038           32           32.0  30.0000  32.4405      183
34466.191570 35397.199355           64           64.0  98.0000  43.0964      169
22650.206457 10834.221345          128          128.0 119.0000  63.0574    

Посмотрим на качество.

In [25]:
! vw --testonly --initial_regressor airbnb-lin-model-1.vw.bin --predictions airbnb-1-predictions.txt airbnb-test.vw

only testing
predictions = airbnb-1-predictions.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
12414.840820 12414.840820            1            1.0  67.0000 178.4219      176
6385.283173 355.725525            2            2.0 149.0000 167.8607      178
5797.542808 5209.802442            4            4.0  50.0000 152.0667      177
11372.974285 16948.405762            8            8.0  55.0000  94.2239      140
11254.900320 11136.826355           16           16.0 349.0000 228.8699      183
7937.500353 4620.100387           32           32.0  73.0000 115.0770      168
6498.084106 5058.667860           64           64.0  52.0000  99.1681       99
14933.415227 23368.746347          128          128.0  49.0000 144.8301      16

In [26]:
! head airbnb-1-predictions.txt

178.421906
167.860687
131.414352
152.066666
24.521982
172.426620
43.975327
94.223877
86.247917
138.389420


In [27]:
with open('airbnb-1-predictions.txt', 'r') as f:
    y_pred = np.array([float(value) for value in f.readlines()])

In [28]:
def read_target_from_vw(vw_object):
    return float(vw_object.split(' ')[0])

with open('airbnb-test.vw', 'r') as f:
    y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])

In [29]:
y_pred

array([178.421906, 167.860687, 131.414352, ..., 129.718094,  50.806786,
       118.676826])

In [30]:
y_expected

array([ 67., 149., 130., ...,  45.,  45., 110.])

In [31]:
from sklearn.metrics import r2_score

In [32]:
r2_score(y_expected, y_pred)

-0.042711300260121154

Да, качество пока не впечатляет. Это связано с тем, что начальные параметры VW достаточно слабые. Однако у VW их достаточно большое количество, что позволяет весьма гибко менять обучаемую модель. Все параметры можно посмотреть, вызвав `vw --help`. Мы рассмотрим несколько важных из них.

* Регуляризация - можно указывать и l1 и l2 через `--l1`, `--l2`
* Размер хеша - можно указать размер признакового пространства после хеширования. Указывается в битах, то есть, указав этот параметр 10, у модели будет 2^10 признаков. Чем больше, чем меньше коллизий и лучше качество. Однако увеличивается потребление памяти и время на расчет. Параметр `--bit_precision`
* Количество эпох - можно указать, сколько раз алгоритму необходимо пройтись по данных в процессе обучения. Для использования также необходимо указать кеш-файл - файл, где будут сохраняться промежуточные результаты. Параметры `--passes` и `--cache_file` соответственно.
* Скорость обучения (learning rate) - можно указать скорость для градиентного спуска. Параметр `--learning_rate`

In [33]:
%%time
! (rm vw.cache || exit 0)
! vw --final_regressor airbnb-lin-model-2.vw.bin airbnb-train.vw --learning_rate 10.0 --bit_precision 22 --passes 2 --cache_file vw.cache

rm: cannot remove 'vw.cache': No such file or directory
final_regressor = airbnb-lin-model-2.vw.bin
Num weight bits = 22
learning rate = 10
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = vw.cache
Reading datafile = airbnb-train.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
247009.000000 247009.000000            1            1.0 497.0000   0.0000      167
124107.208374 1205.416748            2            2.0  42.0000   7.2809      185
71598.113556 19089.018738            4            4.0 198.0000   4.3248        9
53879.962028 36161.810500            8            8.0  49.0000  49.0281       47
35996.759609 18113.557190           16           16.0 160.0000  55.5732       39
22678.413670 9360.067731           32           32.0  35.0000 168.3842      177
25525.760121 28373.106572           64           64.0  70.0000

In [34]:
def calc_r2(predictions_path, answers_path):
    with open(predictions_path, 'r') as f:
        y_pred = np.array([float(value) for value in f.readlines()])
        
    with open(answers_path, 'r') as f:
        y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])
        
    return r2_score(y_expected, y_pred)

In [35]:
! vw --testonly --initial_regressor airbnb-lin-model-2.vw.bin --predictions airbnb-2-predictions.txt airbnb-test.vw

only testing
predictions = airbnb-2-predictions.txt
Num weight bits = 22
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1263.947876 1263.947876            1            1.0  67.0000  31.4480      176
2847.513000 4431.078125            2            2.0 149.0000 215.5663      178
1731.842186 616.171371            4            4.0  50.0000  80.1333      177
4962.735058 8193.627930            8            8.0  55.0000   8.1013      140
16185.806010 27408.876963           16           16.0 349.0000 313.6222      183
9241.604786 2297.403561           32           32.0  73.0000   5.6624      168
6536.176171 3830.747556           64           64.0  52.0000 130.6690       99
12581.899921 18627.623671          128          128.0  49.0000  63.5432      167
10

In [36]:
calc_r2("airbnb-2-predictions.txt", "airbnb-test.vw")

0.36635304933605783

Так гораздо лучше! 

Если сравнивать с моделью PCA + линейная модель и линейная модель на сырых данных, которые мы рассматривали в предыдущий раз, результаты следующие

* Качество выросло с ~0.2 до ~0.37, то есть почти в **2 раза**
* Время обучения упало с 60 и 40 секунд до ~2 , то есть в **35** и  **20** раз соответственно

Очень неплохой результат, но это не все возможности vw. Здесь есть возможность более агресивно использовать метод хеширования - хеширование позволяет нам обрабатывать потенциально любое количество признаков. Почему бы тогда не увеличивать это количество признаков?

Один из способов - это использовать N-граммы, чтобы учитывать подряд идущие слова. Для этого используется параметр `--ngram`.

Или можно комбинировать признаки сами с собой. Другими словами мы будем рассматривать в качестве признаков все возможные пары слов, которые есть в тексте. Например, предложение "Good luck, Mike" будет рассматриваться как ["Good", "luck", "Mike", "Good Mike", "Good luck", "luck Mike"]. Для комбинирования используется `--quadratic`.

Стоит заметить, что изначальных признаков было примерно 100000, а значит новых признаков будет больше чем 100000^2 - огромное число, но хеширование позволяет с этим справится.

In [37]:
%%time
! (rm vw.cache || exit 0)
! vw --final_regressor airbnb-lin-model-3.vw.bin airbnb-train.vw --ngram 3 --learning_rate 10.0 --bit_precision 22 --passes 10 --cache_file vw.cache

Generating 3-grams for all namespaces.
final_regressor = airbnb-lin-model-3.vw.bin
Num weight bits = 22
learning rate = 10
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = vw.cache
Reading datafile = airbnb-train.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
247009.000000 247009.000000            1            1.0 497.0000   0.0000      496
124232.595703 1456.191406            2            2.0  42.0000   3.8399      550
71759.649948 19286.704193            4            4.0 198.0000   2.2944       22
57916.267544 44072.885139            8            8.0  49.0000  32.8372      136
39630.302162 21344.336780           16           16.0 160.0000  38.7350      112
24352.077421 9073.852680           32           32.0  35.0000 156.6530      526
26083.726308 27815.375196           64           64.0  70.0000  72.0418      52

In [38]:
! vw --testonly --initial_regressor airbnb-lin-model-3.vw.bin --predictions airbnb-3-predictions.txt airbnb-test.vw

Generating 3-grams for all namespaces.
only testing
predictions = airbnb-3-predictions.txt
Num weight bits = 22
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
22654.406250 22654.406250            1            1.0  67.0000 217.5138      523
18226.703125 13799.000000            2            2.0 149.0000 266.4691      529
11001.010742 3775.318359            4            4.0  50.0000 120.3091      526
7682.814874 4364.619006            8            8.0  55.0000  81.8713      415
21170.683981 34658.553087           16           16.0 349.0000 431.0936      544
13052.136917 4933.589853           32           32.0  73.0000  73.5171      499
10177.577317 7303.017717           64           64.0  52.0000 127.6307      292
14834.248912 19490.920508          1

In [39]:
calc_r2("airbnb-3-predictions.txt", "airbnb-test.vw")

0.3853230867503401

Еще попытка

In [40]:
%%time
! (rm vw.cache || exit 0)
! vw --final_regressor airbnb-lin-model-4.vw.bin airbnb-train.vw --ngram 2 --learning_rate 40.0 --bit_precision 22 --passes 50 --cache_file vw.cache

Generating 2-grams for all namespaces.
final_regressor = airbnb-lin-model-4.vw.bin
Num weight bits = 22
learning rate = 40
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = vw.cache
Reading datafile = airbnb-train.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
247009.000000 247009.000000            1            1.0 497.0000   0.0000      332
123935.066101 861.132202            2            2.0  42.0000  12.6549      368
75334.963226 26734.860352            4            4.0 198.0000   7.3048       16
52160.772472 28986.581718            8            8.0  49.0000  65.4534       92
34334.568857 16508.365242           16           16.0 160.0000  84.6049       76
22621.677547 10908.786236           32           32.0  35.0000 204.7233      352
25825.555672 29029.433797           64           64.0  70.0000 121.3315      34

In [41]:
! vw --testonly --initial_regressor airbnb-lin-model-4.vw.bin --predictions airbnb-4-predictions.txt airbnb-test.vw

Generating 2-grams for all namespaces.
only testing
predictions = airbnb-4-predictions.txt
Num weight bits = 22
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test.vw
num sources = 1
Enabled reductions: gd, scorer
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
447.284821 447.284821            1            1.0  67.0000  88.1491      350
3415.171463 6383.058105            2            2.0 149.0000 228.8940      354
1819.696119 224.220776            4            4.0  50.0000  61.1000      352
3070.560905 4321.425690            8            8.0  55.0000  25.7257      278
21306.618166 39542.675428           16           16.0 349.0000 362.9811      364
12854.457433 4402.296700           32           32.0  73.0000  91.6448      334
8183.423551 3512.389669           64           64.0  52.0000  83.8710      196
13523.373159 18863.322766          128       

In [42]:
calc_r2("airbnb-4-predictions.txt", "airbnb-test.vw")

0.4390452357106146

Таким образом, манипулируя многочисленными параметрами vw можно быстро обучать модели на огромных массивах данных с хорошим качеством!

Стоит отметить, что хеширование - это не уникальная особенность vw. Инструменты для его использования есть в других инструментах, например в sklearn или Spark.

Для примера попробуем обучить модель sklearn с использованием хеширования.

In [43]:
from sklearn.feature_extraction.text import HashingVectorizer

In [44]:
vectorizer = HashingVectorizer(n_features=2**10)

In [45]:
X_hash = vectorizer.fit_transform(X)

In [46]:
X_hash

<98584x1024 sparse matrix of type '<class 'numpy.float64'>'
	with 7780851 stored elements in Compressed Sparse Row format>

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_hash, Y, test_size=0.33, random_state=100)

In [48]:
%%time

from sklearn.linear_model import Ridge
regressor = Ridge(solver='sparse_cg').fit(X_train, y_train)

CPU times: user 675 ms, sys: 11.7 ms, total: 687 ms
Wall time: 686 ms


In [49]:
y_pred = regressor.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.26486168788999587
